## textから指定されたセクションを抽出する

In [24]:
import fitz  # PyMuPDF
import re
from pathlib import Path
import pandas as pd
import requests
from io import BytesIO



In [25]:
#setting
_MAX_PARAGRAPH = 30
_MIN_LENGTH = 50
_MAX_KYOKUN_LEN = 400  
K = 20  # kmeansのクラスタ数
_REP_NUM = 10  # 各クラスタの代表数



In [26]:

# 1. PDF -> テキスト結合
def extract_text_all(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    return "\n".join(texts)


In [27]:
"""
HEADING_PATTERNS = [
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓|提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓|提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓・提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓・提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓•提言)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓•提言)',
    r'(教訓・提言)',

    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(提言・教訓)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(提言・教訓)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(提言•教訓)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(提言•教訓)',
    r'(提言・教訓)',
    
    r'(JICAへの教訓)',
    r'(JICA への教訓)',

]
"""

"\nHEADING_PATTERNS = [\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\\.\\-．、.][0-9０-９一二三四五六七八九十]+)*[\\s\u3000]*(教訓|提言)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]*[\\s\u3000]*(教訓|提言)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\\.\\-．、.][0-9０-９一二三四五六七八九十]+)*[\\s\u3000]*(教訓・提言)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]*[\\s\u3000]*(教訓・提言)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\\.\\-．、.][0-9０-９一二三四五六七八九十]+)*[\\s\u3000]*(教訓•提言)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]*[\\s\u3000]*(教訓•提言)',\n    r'(教訓・提言)',\n\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\\.\\-．、.][0-9０-９一二三四五六七八九十]+)*[\\s\u3000]*(提言・教訓)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]*[\\s\u3000]*(提言・教訓)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\\.\\-．、.][0-9０-９一二三四五六七八九十]+)*[\\s\u3000]*(提言•教訓)',\n    r'^\\s*[0-9０-９IVX一二三四五六七八九十]*[\\s\u3000]*(提言•教訓)',\n    r'(提言・教訓)',\n    \n    r'(JICAへの教訓)',\n    r'(JICA への教訓)',\n\n]\n"

In [28]:
HEADING_PATTERNS = [
    r'^\s*[0-9０-９IVX一二三四五六七八九十]+(?:[\.\-．、.][0-9０-９一二三四五六七八九十]+)*[\s　]*(教訓)',
    r'^\s*[0-9０-９IVX一二三四五六七八九十]*[\s　]*(教訓)',
    r'(JICAへの教訓)',
    r'(JICA への教訓)',

]

In [29]:
import re

def is_noise_paragraph(text: str) -> bool:
    s = text.strip()

    # 1. まず改行を消した短さチェック用文字列
    s_compact = s.replace("\n", "").strip()

    # 1. 極端に短い
    if len(s_compact) < 8:
        return True

    # 2. 各行が「提言」「教訓」「結論」などだけの行ばかり → ノイズ
    lines = [l.strip() for l in s.splitlines() if l.strip()]
    if lines:
        if all(re.fullmatch(r'[・･\-\.\s　]*(提言|教訓|結論)[・･\-\.\s　]*', l) for l in lines):
            return True

    # 3. 全体で「提言」「教訓」「結論」の繰り返しだけ
    compressed = re.sub(r'[・･\-\.\s　\n]+', '', s)
    if re.fullmatch(r'(提言|教訓|結論)+', compressed):
        return True

    # 4. 脚注っぽい（数字+短いテキスト）
    if re.match(r'^[0-9０-９]+\s', s_compact) and len(s_compact) < 25:
        return True

    # 5. 「主要計画／実績比較」など表見出しブロック
    #   - 主要計画 を含み、
    #   - 「実績比較」か「項  目」「計  画」「実  績」など典型的なカラム名が並んでいる
    if "主要計画" in s:
        if ("実績比較" in s
            or "項  目" in s
            or "計  画" in s
            or "実  績" in s):
            return True

    # 6. 明らかに表形式ぽい：句点がなく、改行だらけ、名詞だけ
    if "。" not in s and len(lines) >= 3:
        # かなり強めに表・見出し臭いものは弾く
        header_keywords = ["項  目", "計  画", "実  績", "指標", "区分"]
        if any(k in s for k in header_keywords):
            return True

    return False

In [30]:
import re

def is_heading_line(s: str) -> bool:
    """数字やローマ数字＋区切り記号＋短いテキスト（句点なし）を見出し候補とみなす。"""
    s = s.strip()
    if "。" in s:
        return False
    return bool(re.match(
        r'^[\s　]*[0-9０-９IVX一二三四五六七八九十]+(?:[\.．\-、][0-9０-９一二三四五六七八九十]+)*[\s　]+.{1,40}$',
        s
    ))


def find_kadai_section(full_text: str) -> str | None:
    lines = full_text.splitlines()

    # 教訓・提言セクション開始を探す
    start_idx = None
    for i, line in enumerate(lines):
        normalized = line.strip()
        for pat in HEADING_PATTERNS:
            if re.search(pat, normalized):
                start_idx = i
                break
        if start_idx is not None:
            break
    if start_idx is None:
        return None

    # 終了条件：
    #  (1) 「以上」「以 上」「以　上」「(以上)」「（以 上）」など
    #  (2) 「コラム」や「主要計画」を含む行
    #  (3) 別の見出し行
    end_idx = len(lines)
    END_LINE_PAT = r'^[\s　]*[（(]?\s*以[\s　]*上\s*[)）]?[\s　]*$'

    for j in range(start_idx + 1, len(lines)):
        line = lines[j].strip()

        # (1) 以上系
        if re.search(END_LINE_PAT, line):
            end_idx = j
            break

        # --- 修正版: 主要計画（全角・空白・スラッシュなど含む） ---
        if re.search(r'主[\s　]*要[\s　]*計[\s　]*画', line):
            end_idx = j
            break

        # --- 修正版: 主要計画（全角・空白・スラッシュなど含む） ---
        if re.search(r'ノンスコア', line):
            end_idx = j
            break

        # --- 修正版: 主要計画（全角・空白・スラッシュなど含む） ---
        if re.search(r'コ[\s　]*ラ[\s　]*ム', line):
            end_idx = j
            break

    section_lines = lines[start_idx:end_idx]
    return "\n".join(section_lines)

In [31]:
import re

def is_page_number_line(s: str) -> bool:
    s = s.strip()
    # 完全に数字だけ / 前後に-や()だけの数字 → ページ番号とみなす
    if re.fullmatch(r'[0-9０-９]+', s):
        return True
    if re.fullmatch(r'[-–－\-（(]?\s*[0-9０-９]+\s*[-–－\-）)]?', s):
        return True
    return False


def split_into_paragraphs(section_text: str) -> list[str]:
    """
    セクションテキストを「行」ではなく「セクション全体 1 つ」にまとめる。
    - ページ番号行を削除
    - 残りの行をすべて結合して 1 つのテキストにする
    """

    # 1) ページ番号行を削除
    cleaned_lines = []
    for line in section_text.splitlines():
        if is_page_number_line(line):
            continue
        cleaned_lines.append(line)

    # 2) 行間の改行をすべて潰す（空白も含めて削る）
    cleaned_text = "\n".join(cleaned_lines)
    # 改行＋前後の空白を全削除（日本語はスペースなくても文として自然なので、つなげてしまう）
    merged_text = re.sub(r'\s*\n\s*', '', cleaned_text).strip()

    if not merged_text:
        return []

    # セクション全体を 1 つの「段落」として返す
    return [merged_text]

In [32]:
def load_pdf_text(source: str) -> str:
    """
    source が
      - http(s) で始まる場合: Web上のPDFとして取得
      - それ以外: ローカルパスとして扱う
    全ページのテキストを結合して返す。
    """
    if source.startswith("http://") or source.startswith("https://"):
        # URLからPDFを取得
        resp = requests.get(source, timeout=30)
        resp.raise_for_status()
        pdf_bytes = resp.content
        doc = fitz.open(stream=BytesIO(pdf_bytes), filetype="pdf")
    else:
        # ローカルファイルとして開く
        doc = fitz.open(source)

    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    doc.close()
    return "\n".join(texts)

In [33]:
import re

def clean_kyokun_text(s: str) -> str:
    """
    教訓・提言テキストのクリーニング:
      - 先頭の「教訓」「提言」「JICAへの教訓」などを削る
      - 先頭のコロン・箇条書き記号を削る
      - 末尾の「…全景」などキャプションっぽい断片を簡易的に削る
      - 空白の正規化 など
    """
    t = s.strip()

    # 1) 先頭の JICA への教訓ヘッダ
    #    例: "JICA への教訓：- 予防接種拡大計画を改善するためには…"
    t = re.sub(
        r'^(JICA\s*への\s*)?(教訓|提言)\s*[:：\-‒‐・･．\.\)）]*',
        '',
        t
    )

    # 2) 素の「教訓」「提言」＋記号だけのヘッダ
    #    例: "教訓•アウトプット…" / "提言: 〜"
    t = re.sub(
        r'^(教訓|提言)\s*[•◦\-‒‐–—･・:：．\.\)）]*',
        '',
        t
    )

    # 3) それでも残る先頭のコロン・箇条書き記号
    #    例: "：- 予防接種…" / "：‒ 鉄分の問題は…"
    t = re.sub(
        r'^[\s　]*[•◦\-‒‐–—･・:：]+',
        '',
        t
    )

    t = t.strip()

    # 4) ごく短いキャプションを末尾から落とす（今回の「全景」対策）
    #    例: "。ミューンズコーズウェイ全景" を最後の "。" までに切る
    cap_m = re.search(r'。[^。]{0,20}(全景|写真|図|地図|Map)$', t)
    if cap_m:
        # 最後の句点までを残す
        last_period = t.rfind('。')
        if last_period != -1:
            t = t[: last_period + 1]

    # 5) 余計な空白を1つに
    t = re.sub(r'\s+', ' ', t)

    return t.strip()

In [34]:
import re

# どこに出てきても「ここからが教訓ブロックっぽい」とみなすトリガー
HEADING_IN_PARAGRAPH_PAT = re.compile(
    r'(教訓)',
    re.IGNORECASE
)

# 先頭にある「番号＋教訓/提言/結論」ヘッダを削るパターン
HEADER_PAT = re.compile(
    r'^'
    r'(第?\s*[0-9０-９一二三四五六七八九十IVX]+'
    r'(?:[\.\-．、][0-9０-９一二三四五六七八九十IVX]+)*)?'  # 任意の番号(4.3, IV-2 など)
    r'\s*'
    r'(教訓|提言|教訓・提言|教訓･提言|結論及び教訓・提言|結論)'
)

def normalize_kyokun_paragraph(p: str) -> str:
    """
    教訓・提言段落を「本文だけ」に正規化するための一本化した関数。

    1) 段落中のどこかに出てくる '教訓/提言/結論' を探し、
       見つかったらそこから先だけを残す。
    2) その結果の先頭に「番号＋教訓/提言/結論」ヘッダがあれば削る。
    """
    s = p.strip()

    # ① 段落途中に教訓/提言/結論が紛れ込んでいる場合、その位置から切り出す
    m = HEADING_IN_PARAGRAPH_PAT.search(s)
    if m:
        s = s[m.start():].lstrip()

    # ② 先頭に残っている「○.○ 教訓」「教訓4.1」などを削る
    m2 = HEADER_PAT.match(s)
    if m2:
        s = s[m2.end():].lstrip()

    return s

In [35]:
import re

# 例: "4.1 結論", "4.2 提言", "II-3 教訓" など
INTERNAL_HEADING_SPLIT_PAT = re.compile(
    r'(?=('
    r'[0-9０-９一二三四五六七八九十IVX]+'
    r'(?:[\.\-．、][0-9０-９一二三四五六七八九十IVX]+)*'
    r'\s*(?:教訓|提言|結論)'
    r'))'
)

def explode_internal_headings(paras: list[str]) -> list[str]:
    """
    1つの段落の中に「4.1 結論」「4.2 提言」「4.3 教訓」など
    複数の見出しが入っている場合、それぞれを別段落に分割する。
    """
    out: list[str] = []

    for p in paras:
        s = p.strip()
        if not s:
            continue

        matches = list(INTERNAL_HEADING_SPLIT_PAT.finditer(s))

        # 見出しが0 or 1回 → そのまま
        if len(matches) <= 1:
            out.append(s)
            continue

        # 見出し位置を取得
        idxs = [m.start() for m in matches]

        # 先頭にゴミがある場合（見出しより前のテキスト）も一応残しておく
        if idxs[0] > 0:
            head0 = s[:idxs[0]].strip()
            if head0:
                out.append(head0)

        idxs.append(len(s))

        # 各見出しから次の見出し直前までを1チャンクとして切り出す
        for i in range(len(matches)):
            chunk = s[idxs[i]:idxs[i+1]].strip()
            if chunk:
                out.append(chunk)

    return out

In [36]:

SENT_SPLIT_PAT = re.compile(r'(?<=[。．！？])\s*')

def split_long_kyokun(text: str,
                      max_len: int = _MAX_KYOKUN_LEN) -> list[str]:
    """
    教訓テキストが長すぎる場合に、「文っぽい単位」で分割しつつ、
    それでも長い場合は文を束ねて max_len 以内のチャンクにまとめる。
    """

    text = text.strip()
    if not text:
        return []

    # 1) 文末記号ベースで素朴に分割
    sentences = [s for s in SENT_SPLIT_PAT.split(text) if s]

    # 文レベルで分割しても十分短いなら、そのまま返す
    if len(text) <= max_len or len(sentences) <= 1:
        return [text]

    chunks: list[str] = []
    buf = ""

    for sent in sentences:
        # 念のため余計な空白除去
        sent = sent.strip()
        if not sent:
            continue

        # バッファにまだ入るなら貯める
        if len(buf) + len(sent) <= max_len:
            buf += sent
        else:
            # いったん確定
            if buf:
                chunks.append(buf)
            buf = sent

    if buf:
        chunks.append(buf)

    # まだ異常に長いチャンクがあれば、さらに接続詞で切るなどを検討してもよい
    # （必要になったらここに「、しかし」「、一方」などで2段目の split を追加）

    return chunks

In [37]:
def extract_kadai_paragraphs(source: str,
                             project_id: str | None = None) -> pd.DataFrame:

    full_text = load_pdf_text(source)
    kadai_section = find_kadai_section(full_text)

    if kadai_section is None:
        print(f"[WARN] 教訓・提言セクションが見つからない: {source}")
        return pd.DataFrame(columns=["project_id", "para_id", "text", "source"])

    # STEP 1: ざっくり段落分割
    paras = split_into_paragraphs(kadai_section)

    # STEP 1.5: 段落内部の「4.1 結論」「4.2 提言」「4.3 教訓」などで分割
    paras = explode_internal_headings(paras)

    cleaned_paras: list[str] = []

    for p in paras:
        raw = p.strip()
        if not raw:
            continue

        # ---- ここで「結論だけのパラグラフ」を落とす ----
        heading_type = None
        m = INTERNAL_HEADING_SPLIT_PAT.search(raw)
        if m:
            # 見出しっぽい部分だけ切り出して、その中のキーワードを見る
            head_sub = raw[m.start(): m.start() + 20]  # 20文字くらい前後を見れば十分
            m2 = re.search(r'(教訓|提言|結論)', head_sub)
            if m2:
                heading_type = m2.group(1)

        # 「4.1 結論 …」みたいなチャンクはここで捨てる
        if heading_type == "結論" or heading_type == "提言":
            continue
        # ---------------------------------------------

        # STEP 2: 見出し削り（教訓/提言などを落として本文化）
        core = normalize_kyokun_paragraph(raw)

        # STEP 3: 細かいクリーニング
        cleaned = clean_kyokun_text(core)

        # STEP 4: ノイズ除去
        if not cleaned.strip():
            continue
        if is_noise_paragraph(cleaned):
            continue

        # ★★ NEW: 長すぎる教訓は「文チャンク」に分割してから追加 ★★
        for chunk in split_long_kyokun(cleaned):
            if chunk.strip():
                cleaned_paras.append(chunk)

    if not cleaned_paras:
        print(f"[WARN] セクションあるが段落なし: {source}")
        return pd.DataFrame(columns=["project_id", "para_id", "text", "source"])

    if project_id is None:
        project_id = Path(source).name

    df = pd.DataFrame({
        "project_id": project_id,
        "para_id": range(len(cleaned_paras)),
        "text": cleaned_paras,
        "source": source,
    })

    return df

In [38]:
except_file_list = [
    # フォーマットが特殊
    'https://www2.jica.go.jp/ja/evaluation/pdf/2010_VNXI-3_4_f.pdf'
]

In [39]:
# dirにあるファイルを一覧で取得する
dir = Path("../pdf_text/")
file_list = [str(f) for f in dir.glob("*.txt")]

#target_urls = df["file"].dropna().unique().tolist()
#target_urls = target_urls[0:200]  # テスト用

#target_urls = ["https://www2.jica.go.jp/ja/evaluation/pdf/2010_C01-P160_4_f.pdf"]
df_out = pd.DataFrame()
for i,url in enumerate(file_list):
    if url in except_file_list:
        print(f"=== {i} {url} SKIPPED ===")
        continue
    df_kadai = extract_kadai_paragraphs(url)
    print(f"=== {i} {url} ===")
    df_out = pd.concat([df_out, df_kadai], ignore_index=True)


=== 0 ../pdf_text/2010_0200600_4_f.txt ===
=== 1 ../pdf_text/2010_0202100_4_f.txt ===
[WARN] セクションあるが段落なし: ../pdf_text/2010_0202700_4_f.txt
=== 2 ../pdf_text/2010_0202700_4_f.txt ===
=== 3 ../pdf_text/2010_0211100_4_f.txt ===
[WARN] 教訓・提言セクションが見つからない: ../pdf_text/2010_0300500_4_f.txt
=== 4 ../pdf_text/2010_0300500_4_f.txt ===
=== 5 ../pdf_text/2010_0300700_4_f.txt ===
=== 6 ../pdf_text/2010_0304000_4_f.txt ===
=== 7 ../pdf_text/2010_0306300_4_f.txt ===
=== 8 ../pdf_text/2010_0307400_4_f.txt ===
[WARN] セクションあるが段落なし: ../pdf_text/2010_0308100_4_f.txt
=== 9 ../pdf_text/2010_0308100_4_f.txt ===
=== 10 ../pdf_text/2010_0309700_4_f.txt ===
=== 11 ../pdf_text/2010_0400200_4_f.txt ===
=== 12 ../pdf_text/2010_0400300_4_f.txt ===
[WARN] セクションあるが段落なし: ../pdf_text/2010_0400900_4_f.txt
=== 13 ../pdf_text/2010_0400900_4_f.txt ===
=== 14 ../pdf_text/2010_0401100_4_f.txt ===
=== 15 ../pdf_text/2010_0401200_4_f.txt ===
=== 16 ../pdf_text/2010_0402400_4_f.txt ===
=== 17 ../pdf_text/2010_0406300_4_f.txt =

In [40]:
# クレンジング
import re

def is_noise_paragraph(text: str) -> bool:
    s = str(text).strip()
    s_compact = s.replace("\n", "").strip()

    # 1. 極端に短い
    if len(s_compact) < 8:
        return True

    # 2. 教訓・提言・結論だけが並んでる系
    lines = [l.strip() for l in s.splitlines() if l.strip()]
    if lines:
        if all(re.fullmatch(r'[・･\-\.\s　]*(提言|教訓|結論)[・･\-\.\s　]*', l) for l in lines):
            return True

    # 3. 全体でも「提言/教訓/結論」だけ
    compressed = re.sub(r'[・･\-\.\s　\n]+', '', s)
    if re.fullmatch(r'(提言|教訓|結論)+', compressed):
        return True

    # 4. 脚注系（数字＋短文）
    if re.match(r'^[0-9０-９]+\s', s_compact) and len(s_compact) < 25:
        return True

    # 5. 主要計画／実績比較＋典型的カラム名
    if "主要計画" in s:
        if ("実績比較" in s
            or "項  目" in s
            or "計  画" in s
            or "実  績" in s):
            return True

    # 6. 表ヘッダーっぽい（句点なし＋複数行＋カラム語）
    if "。" not in s and len(lines) >= 3:
        header_keywords = ["項  目", "計  画", "実  績", "指標", "区分"]
        if any(k in s for k in header_keywords):
            return True

    return False

# 実際に適用
df_out = df_out[~df_out["text"].apply(is_noise_paragraph)].reset_index(drop=True)

In [41]:
# sourceをfileが一致するように変換
df_out["source"] = df_out["source"].str.split("/").str[-1]
df_out["source"] = df_out["source"].str.replace('.txt', '.pdf')

In [42]:
df = pd.read_csv("../df_check_99.csv")
df["file"] = df["file"].str.split("/").str[-1]
# レーティングを結合
df_out = df_out.merge(
    df[["file", "total_eval","eval_year","分野","region_detail"]],
    left_on="source",
    right_on="file",
    how="left"
).drop(columns=["file"])

In [43]:
# project_idごとの段落数をカラムに追加
df_out["project_para_count"] = df_out.groupby("project_id")["para_id"].transform("count")

# 確認（任意）
df_out[["project_id", "para_id", "project_para_count"]].head()

,project_id,para_id,project_para_count
0,2010_0200600_4_f.txt,0,1
1,2010_0202100_4_f.txt,0,1
2,2010_0211100_4_f.txt,0,2
3,2010_0211100_4_f.txt,1,2
4,2010_0300700_4_f.txt,0,1


In [44]:
# 1) _MAX_PARAGRAPH を超える project_id を特定
bad_projects = (
    df_out.groupby("project_id")["project_para_count"]
          .max()                             # そのプロジェクト内の最大段落数
          .loc[lambda s: s > _MAX_PARAGRAPH] # 上限を超えたもの
          .index
)

# 2) それらの project_id に属する段落を全除外
df_out = df_out[~df_out["project_id"].isin(bad_projects)].reset_index(drop=True)

# N文字以下のテキストは除外する(ページ番号や写真の説明文などを除外するため)
mask = df_out["text"].fillna("").str.len() > _MIN_LENGTH
df_out = df_out[mask]

In [45]:
df_out

,project_id,para_id,text,source,total_eval,eval_year,分野,region_detail,project_para_count
0,2010_0200600_4_f.txt,0,事前評価時に設定された運用効果指標の大半は実施機関でデータ収集がなされていない。実施機関が事...,2010_0200600_4_f.pdf,4.0,2010,運輸交通一般,東アフリカ,1
1,2010_0202100_4_f.txt,0,＜指標設定及び基準値の確認にかかる教訓＞本事業の事業事前評価表においては、運用指標と定性的指...,2010_0202100_4_f.pdf,3.0,2010,上水道,東南アジア,1
2,2010_0211100_4_f.txt,0,本事業では植林の活着率95％以上という非常に高い成果を達成した。侵食を受けやすい黄土が厚く堆...,2010_0211100_4_f.pdf,4.0,2010,環境問題,東アジア,2
3,2010_0211100_4_f.txt,1,厳格な施工監理に加え、地域の農民を事業に巻き込んだことも円滑な実施と効果促進の要因といえる。...,2010_0211100_4_f.pdf,4.0,2010,環境問題,東アジア,2
4,2010_0300700_4_f.txt,0,(1) 道路交通安全啓蒙活動と一体化した交通安全向上への取組の必要性一般的に本事業のような道...,2010_0300700_4_f.pdf,3.0,2010,道路,西アフリカ,1
...,...,...,...,...,...,...,...,...,...
4748,2024_1600418_4_f.txt,0,COVID-19 の感染拡大は、本事業完了直後の2020 年3 月以降、EOs及びCGOs ...,2024_1600418_4_f.pdf,4.0,2024,労働,西アジア,2
4749,2024_1600418_4_f.txt,1,実施機関や関連機関による指標のモニタリング状況を十分に把握し、プロジェクト計画時に、事業完了...,2024_1600418_4_f.pdf,4.0,2024,労働,西アジア,2
4750,2024_201403082_4_f.txt,0,本事業のように提案計画の承認や改定の実施のみを評価指標として設定してしまうことで、実施機関に...,2024_201403082_4_f.pdf,2.0,2024,電力,西アフリカ,2
4751,2024_201403082_4_f.txt,1,・本事業は、電力需要予測、電源開発計画、送電拡張計画、投資計画などを含む25 年間のマスター...,2024_201403082_4_f.pdf,2.0,2024,電力,西アフリカ,2


In [46]:
#df_out['text']=df_out['text'].str.replace('\n', '')

In [47]:
import pandas as pd
#df_out = pd.read_csv('../kadai_text_with_rating.csv')

## BERTで埋め込みベクトル作成

In [48]:
%pip install "protobuf<3.21" --upgrade
%pip install -U "transformers" "sentence-transformers"

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(model_name, device='cuda')

texts = df_out["text"].tolist()
embeddings = model.encode(
    texts,
    batch_size=64,
    normalize_embeddings=True,
    show_progress_bar=True,
    convert_to_numpy=True
)

np.save('../embeddings.npy', embeddings)
df_out.to_csv('../kadai_text_with_rating_kyokun.csv', index=False, encoding='utf-8-sig')

2025-11-23 16:24:51.162858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 16:24:51.245961: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 16:24:51.268181: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-23 16:24:51.806204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

Batches:   0%|          | 0/74 [00:00<?, ?it/s]

: 

: 